In [2]:
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import opinion_lexicon
from nltk.corpus import sentiwordnet as swn

nltk.download("opinion_lexicon")

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /home/wrst/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

- **Ejercicio 1:** Se quiere desarrollar un evaluador de sentimientos basado en el uso de un recursos lingüístico externo, en concreto, las listas de palabras de opinión que tiene NLTK.

In [3]:
opinion1 = "Visceral, stunning and relentless film making. Dicaprio's Herculean, almost purely physical performance" \
            "and Hardy's wide eyed intensity coupled with the almost overwhelming beauty of the landscape - those " \
            "trees, the natural light, the sun peeking through the clouds, rendered the proceedings down to savage" \
            "poetry. A hypnotic, beautiful, exhausting film."

opinion2 = "I saw this film on Friday. For the first 40 minutes involving spoken dialogue they need not have " \
            "bothered. For me the dialogue was totally unintelligible with grunting, southern states drawl, " \
            "and coarse accent that made it impossible to understand what they were saying."

opinion3 = "It was a idiotic film that produces a magnificent fascination."

In [ ]:
def tokenize_text(text):
    sent_tokenized = PunktSentenceTokenizer().tokenize(text)
    word_tokenized = [word_tokenize(s) for s in sent_tokenized]
    return word_tokenized

In [ ]:
def classify(text):
    word_tokenized = tokenize_text(text)
    score = 0
    for s in word_tokenized:
        for w in s:
            if w in opinion_lexicon.positive() and w in opinion_lexicon.negative():
                score += 0
            elif w in opinion_lexicon.positive():
                score += 1
            elif w in opinion_lexicon.negative():
                score -= 1
    if score == 0:
        return "neutro", score
    elif score > 0:
        return "pos", score
    else:
        return "neg", score

In [ ]:
# Opinión 1
print(classify(opinion1))

('pos', 1)


In [ ]:
# Opinión 2
print(classify(opinion2))

('neg', -4)


In [ ]:
# Opinión 3
print(classify(opinion3))

('pos', 1)


---
- **Ejercicio 2:** Se pide lo mismo que en el ejercicio anterior, pero en este caso utilizando SentiWordNet (disponible en NLTK). Esta base de datos proporciona valores positivos y negativos para ciertas palabras en un rango entre -1 y 1. Se puede seguir la misma idea de algoritmo que en el caso anterior, pero hay que tener en cuenta que SentiWordNet nos proporciona puntuaciones para los diferentes sentidos que tiene una palabra. Se puede entonces considerar la puntuación de todos los sentidos de la misma palabra, restando a lo positivo la puntuación negativa. Puede ser interesante que la puntuación global se promedie de acuerdo con el número de sentidos.  Utilizar como entrada las mismas opiniones del ejercicio anterior. ¿El resultado es mejor o peor que el conseguido con el algoritmo del ejercicio 1?

In [ ]:
def classify_swn(text):
    word_tokenized = tokenize_text(text)
    total_score = 0
    for s in word_tokenized:
        for w in s:
            synsets = list(swn.senti_synsets(w))
            word_score = 0
            for synset in synsets:
                word_score += (synset.pos_score() - synset.neg_score())
            if len(synsets) > 0:
                total_score += word_score / len(synsets)
    return total_score

In [ ]:
# Opinión 1
print(classify_swn(opinion1))

0.4219623904464331


In [ ]:
# Opinión 2
print(classify_swn(opinion2))

-0.25598923992673994


In [ ]:
# Opinión 3
print(classify_swn(opinion3))

0.7648809523809523


- **Ejercicio 2.1:** Hacer una variante del ejercicio donde se tengan en cuenta primero la categoría gramatical del token para considerar únicamente los scores de los sentidos que coincidan con la categoría gramatical dada. ¿Ha mejorado el resultado o ha empeorado con respecto a versiones anteriores?

In [ ]:
# Función auxiliar para convertir un
# tag generado por nltk.pos_tag()
# en una categ. gramatical de 
# WordNet
def tag2wn(tag):
    if tag.startswith("N"):
        return wn.NOUN
    elif tag.startswith("V"):
        return wn.VERB
    elif tag.startswith("J"):
        return wn.ADJ
    elif tag.startswith("R"):
        return wn.ADV
    else:
        return None

def classify_swn_gram(text):
    word_tokenized = tokenize_text(text)
    tags = [nltk.pos_tag(s) for s in word_tokenized]
    total_score = 0
    for s in tags:
        for w, t in s:
            synsets = list(swn.senti_synsets(w, tag2wn(t)))
            word_score = 0
            for synset in synsets:
                word_score += (synset.pos_score() - synset.neg_score())
            if len(synsets) > 0:
                total_score += word_score / len(synsets)
    return total_score

In [13]:
# Opinión 1
print(classify_swn_gram(opinion1))

0.5139880952380951


In [14]:
# Opinión 2
print(classify_swn_gram(opinion2))

-0.05469148771028469


In [15]:
# Opinión 3
print(classify_swn_gram(opinion3))

0.7655677655677655


Notamos que la mejora es más bien leve, se sigue detectando